In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date

In [2]:
data_frame = pd.read_csv('AWCustomers.csv')

In [3]:
data_frame.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [4]:
def calculateAge(birthDate):
    birthDate = datetime.strptime(birthDate, "%Y-%m-%d").date()
    today = date.today()
    age = today.year - birthDate.year
    return age

In [5]:
updated_df = data_frame.drop(["Title","FirstName","MiddleName","LastName","Suffix","AddressLine1","AddressLine2","City","StateProvinceName","PostalCode","LastUpdated"],axis=1,inplace=False)
updated_df['Age'] = updated_df['BirthDate'].apply(calculateAge)
updated_df.drop(['BirthDate'], axis=1, inplace=True)

In [6]:
updated_df.head()

,CustomerID,CountryRegionName,PhoneNumber,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,21173,Australia,1 (11) 500 555-0115,Bachelors,Clerical,M,M,1,3,0,1,81916,36
1,13249,Canada,787-555-0136,Partial College,Clerical,M,M,1,2,1,2,81076,51
2,29350,United States,120-555-0111,Bachelors,Clerical,F,S,0,3,0,0,86387,38
3,13503,United Kingdom,1 (11) 500 555-0167,Partial College,Skilled Manual,M,M,1,2,1,2,61481,46
4,22803,Germany,1 (11) 500 555-0113,Partial College,Skilled Manual,M,S,1,1,0,0,51804,48


In [7]:
print(updated_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerID            18361 non-null  int64 
 1   CountryRegionName     18361 non-null  object
 2   PhoneNumber           18361 non-null  object
 3   Education             18361 non-null  object
 4   Occupation            18361 non-null  object
 5   Gender                18361 non-null  object
 6   MaritalStatus         18361 non-null  object
 7   HomeOwnerFlag         18361 non-null  int64 
 8   NumberCarsOwned       18361 non-null  int64 
 9   NumberChildrenAtHome  18361 non-null  int64 
 10  TotalChildren         18361 non-null  int64 
 11  YearlyIncome          18361 non-null  int64 
 12  Age                   18361 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 1.8+ MB
None


In [8]:
updated_df.isnull().sum()

CustomerID              0
CountryRegionName       0
PhoneNumber             0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
Age                     0
dtype: int64

In [9]:
normal_df = updated_df
for col in ['NumberCarsOwned','NumberChildrenAtHome','TotalChildren','YearlyIncome','Age']:
    if normal_df[col].dtype == 'int64':
        normal_df[col] = (normal_df[col] - normal_df[col].min()) / (normal_df[col].max() - normal_df[col].min())

In [16]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = [0.2, 0.4, 0.6, 0.8, 1.0]
normal_df['YearlyIncome'] = pd.cut(x = normal_df['YearlyIncome'], bins = bins, labels = labels, include_lowest = True)
normal_df['Age'] = pd.cut(x = normal_df['Age'], bins = bins, labels = labels, include_lowest = True)
print(normal_df.to_string())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
zscore = normal_df
for col in ['NumberCarsOwned','NumberChildrenAtHome','TotalChildren']:
    zscore[col] =(zscore[col]-zscore[col].mean())/ zscore[col].std()

In [18]:
one_hot_df = pd.get_dummies(zscore, columns=['Education'], prefix="Education")
one_hot_df = pd.get_dummies(one_hot_df, columns=['Occupation'], prefix="Occupation")
one_hot_df = pd.get_dummies(one_hot_df, columns=['Gender'], prefix="Gender")
one_hot_df = pd.get_dummies(one_hot_df, columns=['MaritalStatus'], prefix="MaritalStatus")


one_hot_df.drop(['CustomerID','PhoneNumber'],axis = 1,inplace=True)
display(one_hot_df)

,CountryRegionName,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age,Education_Bachelors,Education_Graduate Degree,Education_High School,...,Education_Partial High School,Occupation_Clerical,Occupation_Management,Occupation_Manual,Occupation_Professional,Occupation_Skilled Manual,Gender_F,Gender_M,MaritalStatus_M,MaritalStatus_S
0,Australia,1,1.892473,-0.594355,0.161337,0.6,0.2,1,0,0,...,0,1,0,0,0,0,0,1,1,0
1,Canada,1,0.798367,1.163248,1.239719,0.6,0.4,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,United States,0,1.892473,-0.594355,-0.917044,0.6,0.4,1,0,0,...,0,1,0,0,0,0,1,0,0,1
3,United Kingdom,1,0.798367,1.163248,1.239719,0.4,0.4,0,0,0,...,0,0,0,0,0,1,0,1,1,0
4,Germany,1,-0.295738,-0.594355,-0.917044,0.4,0.4,0,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18356,United States,0,-0.295738,-0.594355,1.239719,0.4,0.2,0,1,0,...,0,0,0,0,0,1,1,0,1,0
18357,Australia,0,0.798367,-0.594355,-0.917044,0.4,0.2,1,0,0,...,0,0,0,0,0,1,1,0,0,1
18358,United States,0,0.798367,-0.594355,-0.917044,0.4,0.4,0,0,0,...,0,0,0,0,0,1,1,0,0,1
18359,Canada,0,-1.389843,-0.594355,-0.917044,0.6,0.2,0,0,1,...,0,1,0,0,0,0,1,0,0,1


In [19]:
def jaccard_sim(a, b):
    numerator = a.intersection(b)
    denominator = a.union(b)
    return len(numerator)/len(denominator)

In [20]:
print(jaccard_sim(set(one_hot_df['YearlyIncome']), set(one_hot_df['Age'])))

1.0


In [21]:
import numpy as np
from numpy.linalg import norm
A = np.array(one_hot_df['Age'])
B = np.array(one_hot_df['YearlyIncome'])
cosine = np.dot(A,B)/(norm(A)*norm(B))
print(cosine)

0.8299037596437723


Divija 102018056 CSBS3